# 7. Differential abundance

******************************************

## Contents
[1. Overview](#overview)

[2. Setting up your analysis environment](#envana3)

[3. Preparing your data](#dataana3)

[4. Choosing a categorical variable to analyse](#varana3)

[5. Comparing ONLY TWO groups](#compana2)

[6. Comparing MORE THAN TWO groups](#compana3etc)


**********************************

## Overview <a class="anchor" id="overview"></a>

In this section we statistically analyse differences in taxa abundance between groups to produce a list of taxa that differ significantly. This is based on [ANCOM (analysis of composition of microbiomes)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4450248/) package, which accounts for compositional constraints to reduce false discoveries in detecting differentially abundant taxa, while maintaining high statistical power.

The specific method and R package for analysing ANCOM is [AMCOM-BC](https://bioconductor.org/packages/release/bioc/vignettes/ANCOMBC/inst/doc/ANCOMBC.html) (BC = Bias Correction).

>Analysis of Compositions of Microbiomes with Bias Correction (ANCOM-BC) (Lin and Peddada 2020) is a methodology of differential abundance (DA) analysis for microbial absolute abundance data. ANCOM-BC estimates the unknown sampling fractions, corrects the bias induced by their differences among samples, and identifies taxa that are differentially abundant according to the covariate of interest.

`Lin, H. and Peddada, S.D., 2020. Analysis of compositions of microbiomes with bias correction. Nature communications, 11(1), pp.1-11.`

The two key statistical scores generated by ANCOM-BC (and presented in tables/figures in this report) are the **W score** and the **q value**.

**W score**: A beta score was calculated using the ANCOM-BC log-linear (refer to [ANCOM-BC publication](https://www.nature.com/articles/s41467-020-17041-7) for details) model, as well as the standard error (SE) for the beta scores. W = beta/SE.

**q value**: P-values were obtained from a two-sided Z-test using the
W scores. These were adjusted for false positives using the Holm–Bonferroni method.

`Holm, S., 1979. A simple sequentially rejective multiple test procedure. Scandinavian journal of statistics, pp.65-70.`

## Results and figures in this section

Each variable contains a number of groups (e.g. a 'lipid content' variable may have 'low', 'medium' and 'high' groups). **Each groups is compared to the first group, in the order the groups are sorted** (e.g. 'medium' and 'high' will each be compared to 'low'). **Thus the first group is considered the control or baseline group and is not shown in the plots**. By default the groups are sorted by alphabetical order, so the baseline group will reflect this. The baseline group can be changed to a different group on request.

The results differ if there are 2 groups or > 2 groups, due to the nature of pairwise analysis.

If there are 2 groups, the results show:

1. A table of global differential abundance W scores and q values. This contains any taxonomic group where at least one group is significantly different from any of the baseline group.

2. A volcano plot of differentially abundant taxa.

If there are > 2 groups, the results show:

1. A table of global differential abundance W scores and q values.

2. Table of groupwise W scores, i.e. W scores for individual groups (if only 2 groups present, then this table is the same as the global W score table, as there is only 1 W column).

3. A Heat map of W scores, to visualise the results in the above table.

4. Table of groupwise q values.

5. A Heat map of q values, to visualise the results in the above table.

*************************

## 2. Setting up your analysis environment <a class="anchor" id="envana3"></a>

Before you can begin your analysis you need to set up certain requirements, such as setting your working directory and installing/loading required R packges.

### Set your working directory

In [ ]:
setwd("~/anacapa")

### Install and load the required R packages

In [ ]:
library(tidyverse)
library(scales)
library(pheatmap)
library(ggrepel)

Install and load the ANCOMBC and phyloseq packages (ANCOMBC installs a version of phyloseq, so just need to load the library).

In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("ANCOMBC")
library(ANCOMBC)
library(phyloseq)

We also need to install the [microbiome package](https://microbiome.github.io/tutorials/), which is used for its aggregate taxa function

In [ ]:
BiocManager::install("microbiome")
library(microbiome)

Define a set of colours for plotting. Some of these plots have multiple clusters and it's difficult to find eougn contrasting colours to visually separate the clusters. I've developed a set of 25 colours that I've found contrast well, that we can use in the plots for this (and other) sections.

In [ ]:
c25 <- c(
  "dodgerblue2", "#E31A1C", # red
  "green4",
  "#6A3D9A", # purple
  "#FF7F00", # orange
  "black", "gold1",
  "skyblue2", "#FB9A99", # lt pink
  "palegreen2",
  "#CAB2D6", # lt purple
  "#FDBF6F", # lt orange
  "gray70", "khaki2",
  "maroon", "orchid1", "deeppink1", "blue1", "steelblue4",
  "darkturquoise", "green1", "yellow4", "yellow3",
  "darkorange4", "brown"
)

Set the default width and height for plots output on this Notebook. You can modify this as you prefer. Note that every plot in this Notebook is followed by code to output it as a file and this code defines width/height separately from the options below.

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)

****************************

## 3. Preparing your data <a class="anchor" id="dataana1"></a>

This analysis requires, as input, an R object in phyloseq format, which has a specific structure. At minumum this requires a samples table, an ASV count table and a taxonomy table, which are then combined into a single phyloseq object.

First you'll need to provide an ID for your project. This must be the project ID you used in the filtration section. See the previous section for details.

In [ ]:
project_id <- "rbcl"

### Import the samples table

In [ ]:
samples_table <- read.csv("sample_table.csv", header = T)

Have a look at your samples table and variables (metadata). In the previous filtration section we didn't use this information, but when examining diversity indices, etc, the metadata is critical.

In [ ]:
samples_table

### Import the ASV abundance table and taxonomy table

**IMPORTANT: this is the FILTERED data that you exported at the end of the previous filtration section. You must have run that section once (only once is needed) for this and following sections to work**

In [ ]:
filtered_data <- read.csv(paste0(project_id, "_filtered_data.csv"))

Have a look at the top few rows of your data. The first 'ASV' column should contain the ASV IDs, the next columns are the samples, followed by the taxonomy levels.

In [ ]:
head(filtered_data)

### Create the phyloseq objects

In the previous sections we used ampvis2, thus created an ampvis2 type object to work with. In this differential abundance section we're using the ANCOM-BC package, which requires a phyloseq type object instead. This is structured slightly different to an ampvis2 object.
https://joey711.github.io/phyloseq/import-data.html

First we'll need to split the ASV abundance and taxonomy data into separate objects.

The taxonomy table:

In [ ]:
mytaxsplit <- subset(filtered_data, select=c(Kingdom, Phylum, Class, Order, Family, Genus, Species))
rownames(mytaxsplit) <- filtered_data$ASV
mytaxsplit <- as.matrix(mytaxsplit)

The ASV abundance table:

In [ ]:
asvtable <- subset(filtered_data, select=-c(ASV, Kingdom, Phylum, Class, Order, Family, Genus, Species))
rownames(asvtable) <- filtered_data$ASV
asvtable <- as.matrix(asvtable)

### Convert the objects to phyloseq format

Convert the samples table into a structure usable by phyloseq

In [ ]:
samp_phylo <- samples_table
row.names(samp_phylo) <- samples_table$sample.id
samp_phylo <- subset(samp_phylo, select=-c(sample.id))

The three objects (ASV table, taxonomy table, samples table) are then combined for each subgroup as a phyloseq object, on which the differential abundance calculations can be done

In [ ]:
ASV_all <- otu_table(asvtable, taxa_are_rows = T)
TAX_all <- tax_table(mytaxsplit)
SAMP_all <- sample_data(samp_phylo)

physeq_all <- phyloseq(ASV_all, TAX_all, SAMP_all)

You can have a look at the component of the phyloseq object like so:

In [ ]:
physeq_all

************************************

## 4. Choosing a categorical variable to analyse <a class="anchor" id="var3"></a>

In your metadata you'll usually have multiple variables. These need to be analysed individually, by selecting the variable in this section, then running the remaining analysis sections on this chosen variable. You can then re-run the analysis on another variable by returning to this section, changing the variable name, then running again the remaining analysis sections.

**NOTE** This section is for choosing categorical variables only. Since differential abundance requires comparing abundance between 2 or more categories, continuous variables can't be analysed.

You can view your variables as column names in your samples_table:

In [ ]:
colnames(samp_phylo)

Enter the column name of the variable you want to analyse (i.e. change `group <- "Myvariable"` in the below cell to your chosen variable's column name). This has to be exactly the same as the column name, including capitalisation, characters such as underscores, etc.

In [ ]:
group <- "Site"

### Ordering your variable

The plotting done in ampvis2 is done by the [ggplot2](https://ggplot2.tidyverse.org/) package. ggplot [factorises](https://www.datamentor.io/r-programming/factor/) variables and automatically orders them on the plot by alphabetical order.

This can cause your groups to be ordered incorrectly on the plot axes (e.g. a time series may not be plotted sequentially). 

You can manually set the order of your variable here. 

First have a look at how ggplot will order your variable.

In [ ]:
levels(factor(physeq_all@sam_data[[group]]))

If these are in the order you want to see them on your plot axes, nothing needs to be done. If they are in the wrong order you need to order them manually by setting the [**levels**](https://www.datamentor.io/r-programming/factor/).

Choose how you want to order your groups here:

In [ ]:
lev <- c("S1", "S2", "S3", "S4")

To order your variable you need to put **all** the variable levels into the `lev = c(..)`. Make sure each level is in double quotes and separated by a comma.

Then run the following to apply the levels to your data:

In [ ]:
physeq_all@sam_data[[group]] <- factor(physeq_all@sam_data[[group]], levels = lev)

Then you can check the levels of your variable again to see if it is in the correct order

In [ ]:
levels(factor(physeq_all@sam_data[[group]]))

**************************************

## 5. Comparing ONLY TWO groups <a class="anchor" id="compana2"></a>

Because of the way differential abundance is analysed, there is a difference in analysis methods if there are only 2 subgroups or > 2 subgroups.

If there are only 2 subgroups, a volcano plot is created (which represents abundance of subgroup 'A' vs subgroup 'B') but if there are > 2 groups a colcano plot cannot be created, rather heatpams are used to visualise abundance differences between subgroup 'A' vs subgroup 'B' vs subgroup 'C' ... etc. 

You can see how many subgroups your selected variable has by running the below code

In [ ]:
length(unique(physeq_all@sam_data[[group]]))

**If your variable has > 2 subgroups, go to the next section (6. Comparing MORE THAN TWO groups)**

**If your variable has exactly 2 subgroups, continue with this section and skip the next section**

First, choose the taxonomic level you want to plot (Choose from Phylum, Class, Order, Family, Genus, Species)

In [ ]:
taxgroup <- "Family"

Then you need to aggregate your data by your chosen taxonomic group (using the `aggregate_taxa` function from microbiome package) 

In [ ]:
taxdata <- aggregate_taxa(physeq_all, taxgroup)

Then run ANCOMBC on your data (based on your selected variable and taxonomic level)

There are a variety of arguments that can be modified here (or you can run the below code as-is). You can see what each argument is and does in the ANCOMBC manual: https://bioconductor.org/packages/release/bioc/manuals/ANCOMBC/man/ANCOMBC.pdf

In [ ]:
out <- ancombc(phyloseq = taxdata, formula = group, 
              p_adj_method = "holm", zero_cut = 0.90, lib_cut = 1000, 
              group = group, struc_zero = TRUE, neg_lb = TRUE, tol = 1e-5, 
              max_iter = 100, conserve = TRUE, alpha = 0.05, global = TRUE)

**NOTE: you need to have at least 2 samples per subgroup, or ANCOMBC won't run. If some of your subgroups have only 1 sample, consider subsetting your data to remove those subgroups**

### Table of global differentially abundant taxa

The below table shows W score, p value and q value (adjusted p value) for **significantly** (q < 0.05) **global** differentially abundant taxa - i.e., a taxonomic group is shown here if one or more of the groups is significantly different from the baseline group.

In [ ]:
tabDA <- cbind(out$res$W, out$res$p_val, out$res$q_val)
colnames(tabDA) <- c("W", "p value", "q value")
tabDA

### Volcano plot of differentially abundant taxa

Significantly (q value < 0.05) different taxa are coloured red. The horizontal dashed line = q value of 0.05

Pull out the W scores and q values, then merge these

In [ ]:
W <- out$res$W
qval <- out$res$q_val
res2 <- cbind(W, qval)
colnames(res2) <- c("W", "q_val")
res2$Taxa <- row.names(res2)

Need to convert qvals to -log

In [ ]:
# If there are any zeroes, a log conversion will create an infinite value. So need to first convert 0 to NA
res2$q_val[res2$q_val == 0] <- NA
res2$log_q <- -log(res2$q_val)

Now create the volcano plot. This includes a horizontal dashed line at q value of 0.05 - `geom_hline()`, a vertical line at zero - `geom_vline()` - and the significant q values are labelled with the taxa names

As with the plots in the previous sections, you can modify various attributes for this plot, such as the colours for the significant (`"firebrick"`) vs non-significant (`"grey50"`) dot points, the size of the points (`size=6`), the main plot theme (`theme_bw()`) and various other attributes. Feel free to modify as you like.

In [ ]:
p <- ggplot(res2, aes(x=W, y=log_q)) + 
  geom_point(size=6, color = ifelse(res2$q_val < 0.05, "firebrick", "grey50"), alpha = 0.8) + theme_bw() + 
  geom_hline(yintercept=-log(0.05), linetype="dashed", col = "grey50") + 
  geom_vline(xintercept=0, linetype="dashed", col = "grey50") + 
  geom_label_repel(data=subset(res2, q_val < 0.05), size = 4, min.segment.length = 0, point.padding = 0, box.padding = 0.3, aes(label=Taxa), max.overlaps = 15) + theme(axis.text = element_text(size=12), axis.title = element_text(size=14)) + xlab("W") + ylab("-log(q)")
p

Then, as with the other plots, you can export the volcano plot as a pdf or tiff file. 

Remember, you can adjust the plot height and width.

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0("DA_volcano_", group, "_", taxgroup, ".tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0("DA_volcano_", group, "_", taxgroup, ".pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

****************************

## 6. Comparing MORE THAN TWO groups <a class="anchor" id="compana3etc"></a>

First, choose the taxonomic level you want to plot (Choose from Phylum, Class, Order, Family, Genus, Species)

In [ ]:
taxgroup <- "Family"

Then you need to aggregate your data by your chosen taxonomic group (using the `aggregate_taxa` function from microbiome package) 

In [ ]:
taxdata <- aggregate_taxa(physeq_all, taxgroup)

Then run ANCOMBC on your data (based on your selected variable and taxonomic level)

There are a variety of arguments that can be modified here (or you can run the below code as-is). You can see what each argument is and does in the ANCOMBC manual: https://bioconductor.org/packages/release/bioc/manuals/ANCOMBC/man/ANCOMBC.pdf

In [ ]:
out <- ancombc(phyloseq = taxdata, formula = group, 
              p_adj_method = "holm", zero_cut = 0.90, lib_cut = 1000, 
              group = group, struc_zero = TRUE, neg_lb = TRUE, tol = 1e-5, 
              max_iter = 100, conserve = TRUE, alpha = 0.05, global = TRUE)

**NOTE: you need to have at least 2 samples per subgroup, or ANCOMBC won't run. If some of your subgroups have only 1 sample, consider subsetting your data to remove those subgroups**

### Table of global differentially abundant taxa

The below table shows W score, p value and q value (adjusted p value) for **significantly** (q < 0.05) **global** differentially abundant taxa - i.e., a taxonomic group is shown here if one or more of the groups is significantly different from the baseline group.

In [ ]:
out$res_global

### Table of groupwise W scores

This table shows the W scores for each group, effectively showing how different each group is from the baseline group (for your selected taxa).

In [ ]:
out$res$W

### Heat map of W scores

This is a heat map of the W scores in the above table.

First we need to define the colour range and set zero as the centre, so that one colour = > 0 score and the other colour = < 0 scores.

In [ ]:
paletteLength <- 100
colW <- colorRampPalette(c("#D55E00", "white", "#009E73"))(paletteLength)
myBreaks <- c(seq(min(out$res$W), 0, length.out=ceiling(paletteLength/2) + 1), seq(max(out$res$W)/paletteLength, max(out$res$W), length.out=floor(paletteLength/2)))

Then create the heatmap. You can change the font size of the rows (`fontsize_row =`) and columns (`fontsize_col =`)

In [ ]:
p <- pheatmap(as.matrix(out$res$W), cluster_rows = F, cluster_cols = F, color=colW, fontsize_col = 14, fontsize_row = 12, breaks=myBreaks, cellwidth = 30, border_color = "grey40")

p

If there are too many taxa (i.e. the plot has too many rows) you can modify it by running the following code (or skip to outputting heatmap as pdf and tiff, if you want to plot all taxa):

First, select which rows you want to plot. This can be a range (e.g. rows 1 - 10 would be `c(1:5)`, rows 10 - 20 would be `c(10:20)`) or a selection of taxa (e.g. taxa on rows 1, 5, 7, 12 would be `c(1, 5, 7, 12))

In [ ]:
subw <- c(1:5)

Then run the plot again

In [ ]:
p <- pheatmap(as.matrix(out$res$W[subw,]), cluster_rows = F, cluster_cols = F, color=colW, fontsize_col = 14, fontsize_row = 12, breaks=myBreaks, cellwidth = 30, border_color = "grey40")

p

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0("DA_heatmap_w_", group, "_", taxgroup, ".tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0("DA_heatmap_w_", group, "_", taxgroup, ".pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

### Table of groupwise q values

You can also generate a table and heatmap of q values (adjusted p values) to examine the signifcance of the W scores

In [ ]:
out$res$q_val

Now create the heatmap (we don't need to centre the colours this time). Non-significant results are coloured grey.

**NOTE:** The heatmap will error if there are all zero qval scores, as seen in the above table. This is rare, but can be seen when all the differences are highly significant. If all scores = 0 then there are no differences to colour in the heatmap (some differences are required)

In [ ]:
# Change non-sig results to 'NA'
out$res$q_val[out$res$q_val > 0.05] <- NA
# Set the colour palette
col <- colorRampPalette(c("#009E73", "yellow"))(100)
# Create the heatmap
p <- pheatmap(as.matrix(out$res$q_val), cluster_rows = F, cluster_cols = F, col = col, fontsize_col = 12, fontsize_row = 8, cellwidth = 30, border_color = "grey40")
p

As with the W score heatmap, you can also subset out a group of taxa to plot. Select the taxa rows below:

In [ ]:
subw <- c(1:10)

Then run the plot again

In [ ]:
p <- pheatmap(as.matrix(out$res$q_val[subw,]), cluster_rows = F, cluster_cols = F, col = col, fontsize_col = 12, fontsize_row = 8, cellwidth = 30, border_color = "grey40")
p

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0("DA_heatmap_q_", group, "_", taxgroup, ".tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0("DA_heatmap_q_", group, "_", taxgroup, ".pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

**This is the end of the analysis pipeline**